In [1]:
from pyroSAR import identify
from pyroSAR.snap.auxil import parse_recipe, parse_node
from pyroSAR.snap import geocode
from spatialist import Vector
from pyroSAR import Archive

from IPython.display import clear_output

import pandas as pd
import logging
import os

# detailed debug & log info
logging.basicConfig(level=logging.INFO)
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)
logging.basicConfig(level=logging.DEBUG)

/tmp/ipykernel_182036/1916172898.py:9: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
workspace_directory = r'/home/ubuntu/Tryggvi/my-awesome-masters-project' #VM
# workspace_directory = r"C:/Users/tryggvisi/Documents/my-awesome-masters-project" #vinna
# workspace_directory = r"C:/Users/trygg/Documents/Master_project" #home
os.chdir(workspace_directory)

In [3]:
#náð í listann af granule ID

granule_list = r'Data/Vindefjallen_data/Vindefjallen_cleaning/Vindefjallen_Granule_processing/list_of_slc_granules_2023.csv'
#import csv
granules = pd.read_csv(granule_list)
#granule_id column to list
granule_id = granules['granule_ID'].tolist()
granule_id

['S1A_IW_SLC__1SDV_20230326T053811_20230326T053838_047811_05BE83_3E60',
 'S1A_IW_SLC__1SDV_20230325T161511_20230325T161538_047803_05BE49_44F8',
 'S1A_IW_SLC__1SDV_20230311T163132_20230311T163159_047599_05B766_9C49',
 'S1A_IW_SLC__1SDV_20230306T162321_20230306T162348_047526_05B4F9_F2AC',
 'S1A_IW_SLC__1SDV_20220316T163126_20220316T163153_042349_050C83_200D',
 'S1A_IW_SLC__1SDV_20220314T053007_20220314T053035_042313_050B38_3653',
 'S1A_IW_SLC__1SDV_20220314T052943_20220314T053010_042313_050B38_6F1B',
 'S1A_IW_SLC__1SDV_20220309T052155_20220309T052223_042240_0508CF_95FB',
 'S1A_IW_SLC__1SDV_20220304T163126_20220304T163153_042174_05068B_A8BE',
 'S1A_IW_SLC__1SDV_20210503T162311_20210503T162338_037726_0473C7_E72E',
 'S1A_IW_SLC__1SDV_20210417T053801_20210417T053828_037486_046B5D_2B21',
 'S1A_IW_SLC__1SDV_20210412T052938_20210412T053005_037413_0468E0_B2A5',
 'S1A_IW_SLC__1SDV_20210328T162310_20210328T162337_037201_046194_6525',
 'S1A_IW_SLC__1SDV_20210326T052149_20210326T052217_037165_04604D

In [4]:
#create a dictionary with granule_id as key and path to granule as value. This is used for the geocode function to geocode the granules in the dictionary
#output directory
granule_directory = '/home/ubuntu/Tryggvi/Data/asf_slc_granules_dl'

#The output directory for the processed granules. Geocode will add 4 rasters and 1 xml file to each directory
# output_directory = '/home/ubuntu/Tryggvi/Data/pyrosar_processed_granules'
# output_directory = '/home/ubuntu/Tryggvi/Data/pyrosar_processed_granules_500' # fyrir ML 500
output_directory = '/home/ubuntu/Tryggvi/Data/pyrosar_processed_granules_1000' # fyrir ML 1000

#the output directory to be used after processing for the clipped granules. Each directory will contain 4 clipped rasters
# clip_output_directory = '/home/ubuntu/Tryggvi/Data/processed_granules_clip'
# clip_output_directory = '/home/ubuntu/Tryggvi/Data/processed_granules_clip_500' # fyrir ML 500
clip_output_directory = '/home/ubuntu/Tryggvi/Data/processed_granules_clip_1000' # fyrir ML 1000

ML_NR = '_1000'

path_to_granule = [granule_directory + '/' + i+'.zip' for i in granule_id ]
path_to_output_directory = [output_directory + '/' +str(i)+'_'+ granule_id[i][17:21] + '-' + granule_id[i][21:23] + '-' + granule_id[i][23:25] + ML_NR for i in range(len(granule_id))]
path_to_clipped_output_directory = [clip_output_directory + '/' +str(i)+'_'+ granule_id[i][17:21] + '-' + granule_id[i][21:23] + '-' + granule_id[i][23:25] + '_clip' + ML_NR for i in range(len(granule_id))]

#create dictionary with path_to_granule and path_to_output_directory as values and key is range of granules
granule_paths_dict = {}
for i in range(len(granule_id)):
    granule_paths_dict[i] = { 
        "path_to_granule": path_to_granule[i], 
        'path_to_output_directory': path_to_output_directory[i],
        'path_to_clipped_output_directory': path_to_clipped_output_directory[i]
    }
granule_paths_dict[1]


{'path_to_granule': '/home/ubuntu/Tryggvi/Data/asf_slc_granules_dl/S1A_IW_SLC__1SDV_20230325T161511_20230325T161538_047803_05BE49_44F8.zip',
 'path_to_output_directory': '/home/ubuntu/Tryggvi/Data/pyrosar_processed_granules_1000/1_2023-03-25_1000',
 'path_to_clipped_output_directory': '/home/ubuntu/Tryggvi/Data/processed_granules_clip_1000/1_2023-03-25_clip_1000'}

### Process granule with geocode

In [20]:
for i in range(len(granule_paths_dict)):
    input_path = granule_paths_dict[i]['path_to_granule']
    output_path = granule_paths_dict[i]['path_to_output_directory']
    print("Processing granule number " , i)
    print(input_path)
    print(output_path)
    geocode(
        infile = input_path,
        outdir = output_path,
        t_srs='EPSG:3006', #SWEREF
        polarizations = ['VV', 'VH'],
        scaling = 'linear',
        removeS1ThermalNoise = True,
        terrainFlattening = True,

        demName = 'Copernicus 30m Global DEM',
        export_extra =['localIncidenceAngle', 'layoverShadowMask'],
        alignToStandardGrid = True, #
        speckleFilter = 'Refined Lee',
        refarea = 'gamma0',
        rlks = 271,
        azlks = 72,
        spacing = 1000,
        clean_edges = True,
        test = False
    )
    clear_output(wait=True)



Processing granule number  56
/home/ubuntu/Tryggvi/Data/asf_slc_granules_dl/S1B_IW_SLC__1SDV_20170221T163813_20170221T163841_004401_007A7F_64A1.zip
/home/ubuntu/Tryggvi/Data/pyrosar_processed_granules_1000/56_2017-02-21_1000


INFO:pyroSAR.snap.auxil:executing node sequences..
INFO:pyroSAR.snap.auxil:Calibration -> ThermalNoiseRemoval
INFO:pyroSAR.snap.auxil:TOPSAR-Deburst
INFO:pyroSAR.snap.auxil:Apply-Orbit-File
INFO:pyroSAR.snap.auxil:Multilook
INFO:pyroSAR.snap.auxil:Terrain-Flattening
INFO:pyroSAR.snap.auxil:Speckle-Filter
INFO:pyroSAR.snap.auxil:Terrain-Correction
INFO:pyroSAR.snap.auxil:cleaning image edges and converting to GeoTIFF
INFO:pyroSAR.snap.util:deleting temporary files
INFO:pyroSAR.snap.util:done


# Clip raster

In [7]:
#create all the required directories for the clipped granules

# for key, folder_path in granule_paths_dict.items():
#     print(folder_path['path_to_clipped_output_directory'])
#     if not os.path.exists(folder_path['path_to_clipped_output_directory']):
#         os.makedirs(folder_path['path_to_clipped_output_directory'])
#         print(f"Folder created successfully for {key}")
#     else:
#         print(f"Folder for {key} already exists.")

/home/ubuntu/Tryggvi/Data/processed_granules_clip_1000/0_2023-03-26_clip_1000
Folder created successfully for 0
/home/ubuntu/Tryggvi/Data/processed_granules_clip_1000/1_2023-03-25_clip_1000
Folder created successfully for 1
/home/ubuntu/Tryggvi/Data/processed_granules_clip_1000/2_2023-03-11_clip_1000
Folder created successfully for 2
/home/ubuntu/Tryggvi/Data/processed_granules_clip_1000/3_2023-03-06_clip_1000
Folder created successfully for 3
/home/ubuntu/Tryggvi/Data/processed_granules_clip_1000/4_2022-03-16_clip_1000
Folder created successfully for 4
/home/ubuntu/Tryggvi/Data/processed_granules_clip_1000/5_2022-03-14_clip_1000
Folder created successfully for 5
/home/ubuntu/Tryggvi/Data/processed_granules_clip_1000/6_2022-03-14_clip_1000
Folder created successfully for 6
/home/ubuntu/Tryggvi/Data/processed_granules_clip_1000/7_2022-03-09_clip_1000
Folder created successfully for 7
/home/ubuntu/Tryggvi/Data/processed_granules_clip_1000/8_2022-03-04_clip_1000
Folder created successfull

In [8]:
import geopandas as gpd
import rasterio
from rasterio.mask import mask
from shapely.geometry import box
from datetime import datetime




# Read the raster data
def clip_raster(raster_file, output_clipped_raster):
    with rasterio.open(raster_file) as src:
        shapefile = "Data/Vindefjallen_data/outputs/shapefiles/study_area.shp"
        # Read the shapefile
        gdf = gpd.read_file(shapefile)
        # Reproject the shapefile to the raster's CRS if they are different
        gdf = gdf.to_crs(src.crs)
        
        # Get the geometry of the shapefile
        geom = gdf.geometry.values[0]
        
        # Clip the raster with the shapefile's geometry
        out_image, out_transform = mask(src, [geom], crop=True)
        out_meta = src.meta.copy()

        # Update metadata for the clipped raster
        out_meta.update({"driver": "GTiff",
                        "height": out_image.shape[1],
                        "width": out_image.shape[2],
                        "transform": out_transform})

        # Write the clipped raster to a new file
        with rasterio.open(output_clipped_raster, "w", **out_meta) as dest:
            dest.write(out_image)
        # Get the current date and time
current_datetime = datetime.now()

# Print the current date and time
print("Current Date and Time:", current_datetime)




Current Date and Time: 2024-03-08 09:26:19.611931


In [9]:
# Loop through each granule in the granule_paths_dict
for i in range(len(granule_paths_dict)):
    # Get the list of files in the output directory of the current granule
    file_list = os.listdir(granule_paths_dict[i]['path_to_output_directory'])
    
    # Loop through each file in the file list
    for j in file_list:
        # Check if the file name contains 'VH_gamma0-rtc.tif'
        if 'VH_gamma0-rtc.tif' in j:
            # Get the path to the raster file
            raster_file = granule_paths_dict[i]['path_to_output_directory'] + '/' + j
            # Get the path to the clipped output raster
            output_clipped_raster = granule_paths_dict[i]['path_to_clipped_output_directory']+ '/' + j
            # Call the clip_raster function to clip the raster
            clip_raster(raster_file, output_clipped_raster)
            # Print the path to the clipped raster
            print('Clipped raster saved to', output_clipped_raster)
        
        # Check if the file name contains '_localIncidenceAngle.tif'
        if '_localIncidenceAngle.tif' in j:
            # Get the path to the raster file
            raster_file = granule_paths_dict[i]['path_to_output_directory'] + '/' + j
            # Get the path to the clipped output raster
            output_clipped_raster = granule_paths_dict[i]['path_to_clipped_output_directory']+ '/' + j
            # Call the clip_raster function to clip the raster
            clip_raster(raster_file, output_clipped_raster)
        
        # Check if the file name contains '_layoverShadowMask.tif'
        if '_layoverShadowMask.tif' in j:
            # Get the path to the raster file
            raster_file = granule_paths_dict[i]['path_to_output_directory'] + '/' + j
            # Get the path to the clipped output raster
            output_clipped_raster = granule_paths_dict[i]['path_to_clipped_output_directory']+ '/' + j
            # Call the clip_raster function to clip the raster
            clip_raster(raster_file, output_clipped_raster)
        
        # Check if the file name contains '_VV_gamma0-rtc.tif'
        if '_VV_gamma0-rtc.tif' in j:
            # Get the path to the raster file
            raster_file = granule_paths_dict[i]['path_to_output_directory'] + '/' + j
            # Get the path to the clipped output raster
            output_clipped_raster = granule_paths_dict[i]['path_to_clipped_output_directory']+ '/' + j
            # Call the clip_raster function to clip the raster
            clip_raster(raster_file, output_clipped_raster)
    

Clipped raster saved to /home/ubuntu/Tryggvi/Data/processed_granules_clip_1000/0_2023-03-26_clip_1000/S1A__IW___D_20230326T053811_VH_gamma0-rtc.tif
Clipped raster saved to /home/ubuntu/Tryggvi/Data/processed_granules_clip_1000/1_2023-03-25_clip_1000/S1A__IW___A_20230325T161511_VH_gamma0-rtc.tif
Clipped raster saved to /home/ubuntu/Tryggvi/Data/processed_granules_clip_1000/2_2023-03-11_clip_1000/S1A__IW___A_20230311T163132_VH_gamma0-rtc.tif
Clipped raster saved to /home/ubuntu/Tryggvi/Data/processed_granules_clip_1000/3_2023-03-06_clip_1000/S1A__IW___A_20230306T162321_VH_gamma0-rtc.tif
Clipped raster saved to /home/ubuntu/Tryggvi/Data/processed_granules_clip_1000/4_2022-03-16_clip_1000/S1A__IW___A_20220316T163126_VH_gamma0-rtc.tif
Clipped raster saved to /home/ubuntu/Tryggvi/Data/processed_granules_clip_1000/5_2022-03-14_clip_1000/S1A__IW___D_20220314T053007_VH_gamma0-rtc.tif
Clipped raster saved to /home/ubuntu/Tryggvi/Data/processed_granules_clip_1000/6_2022-03-14_clip_1000/S1A__IW___